In [33]:
import bs4
from selenium import webdriver
import time
from datetime import datetime
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd
from sys import stdout

In [2]:
wait = 3

In [3]:
# INSTANCE BROWSER
def openBrowser(url, exchange=None):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    import platform
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
#         profile_path = '/home/nomadbitcoin/whatsapp_profiles/' + business 
    elif platform.system() == 'Windows':
        driver_path = r'C:\\Users\\Yan\\Desktop\\brisbane_bot\\webdriver\\win\\chromedriver.exe'
        profile_path = "C:\\Users\\Yan\\Desktop\\TRADING\\ten_minutes_prediction\\browser_profile\\" + 'academiadasapostas'
        print(profile_path)
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
        driver.get(url)
        return True
    except Exception as error:
        print(type(error), error)
        return False

In [4]:
academia_das_apostas = 'https://www.academiadasapostasbrasil.com/'

In [5]:
def showAllEvents():
    ''' clica em vizualizar todos os jogos ocultos, isso pode acontecer mais de uma vez'''
    while True:
        try:
            show_more_games = driver.find_element_by_class_name('footer')
            show_more_games.click()
            time.sleep(wait+3)
        except Exception as error:
            if 'NoSuchElementException' in str(type(error)):
                return True
            else:
                return False #ocorreu algum erro desconhecido

In [36]:
# Salva em CSV os eventos do dia
def saveCSV(data_events, last_datetime):
    '''
        Recebe a lista com os dicionarios dos dados dos eventos, transforma em um dataframe e salva em csv
    '''
    name_file = 'data\\' + last_datetime + '_with_' + str(len(data_events)) + '_events'  + '.csv'
    
    df = pd.DataFrame(data_events)
    df.to_csv(name_file, index = False)
    print('saved file: {}'.format(name_file))
    return True

In [7]:
def getEventsData():
    '''
        Ira pegar os dados dos eventos
    '''
    htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
    events_table = htmlPage.find('table',{'class':'competition-today'})
    event_list = events_table.find_all('tr',{'type':'match'})
    
    global data_events 
    data_events = []
    for event in event_list:
        countrie = event.find('td',{'class':'flag tipsy-active'}).get('original-title')
        timestamp = int(event.find('td',{'class':'hour'}).get('timestamp'))
        date_time = datetime.fromtimestamp(int(event.find('td',{'class':'hour'}).get('timestamp'))).strftime('%d/%m/%y %H:%M:%S')
        status = ''.join(filter(lambda clean_status: clean_status != ' ', (event.find('td',{'class':'status'}).text.replace('\n', ''))))
        home_team = event.find('td',{'class':'team-a'}).find('p').text
        back_team = event.find('td',{'class':'team-b'}).find('p').textback_team = event.find('td',{'class':'team-b'}).find('p').text
        event_link = event.find('td',{'class':'score'}).find('a').get('href')

        event_info = {'timestamp': timestamp, 'status': status, 'countrie': countrie, 'home_team': home_team, 
                 'back_team': back_team, 'event_link': event_link, 'datetime':date_time}

        data_events.append(event_info)

    '''para salvar o arquivo com os dados dos jogos em csv'''
    last_datetime = str(date_time).split(' ')[0].replace('/', '_')
    if saveCSV(data_events, last_datetime):
        return True
    else:
        return False

In [8]:
def get_backPage():
    '''clica para ir para a pagina anterior'''
    try:
        driver.find_element_by_xpath('//span[@class="aa-icon-back date-decrease"]').click()
#         time.sleep(wait)
        return True
    except Exception as error:
        print(type(error), error)
        return False

In [9]:
openBrowser(academia_das_apostas)

C:\Users\Yan\Desktop\TRADING\ten_minutes_prediction\browser_profile\academiadasapostas


True

In [166]:
showAllEvents()

False

39

In [32]:
total_saved = len(os.listdir(r'data'))
# for i in range(total_saved + 9):
#     get_backPage()
    
if getEventsData():
    time.sleep(wait)
    if not get_backPage():
        print('problema em voltar para pagina anterior')
#         break
    else:
        print('{} Páginas salvas'.format(total_saved))
#         count +=1
else: 
    print('problema em pegar os dados dos jogos')

saved file: data\25_02_17_with_761_events.csv
1110 Páginas salvas


In [34]:
get_backPage()

True

In [22]:
total_saved

939

In [ ]:
total_saved = len(os.listdir(r'data'))
count = total_saved
while True:
    stdout.write('\r'+ '\nDados de jogos coletados: {}\n'.format(sum([int(game.split('_')[-2]) for game in os.listdir(r'data')])))
    if showAllEvents():
        time.sleep(wait)
        try:
            if getEventsData():
                time.sleep(wait)
                if not get_backPage():
                    print('problema em voltar para pagina anterior')
                    break
                else:
#                     print('{} Páginas salvas'.format(count))
                    count +=1
            else:
                print('problema em pegar os dados dos jogos')
                break
        except Exception as error:
            if 'UnboundLocalError' in str(type(error)):
                driver.get(academia_das_apostas)
                time.sleep(wait)
                total_saved = len(os.listdir(r'data'))
                for i in range(total_saved + 9):
                    get_backPage()
            else:
                break
                print(type(error), error)
                
    else:
        print('problema em abrir os eventos')
        
        #tenta passar o bug
        try:
            if driver.find_element_by_class_name('footer').text.startswith('+ 2'):
                pass
            else:
                time.sleep(5)
                driver.find_element_by_tag_name('html').send_keys(Keys.END)
                driver.find_element_by_tag_name('html').send_keys(Keys.PAGE_UP)
                driver.find_element_by_tag_name('html').send_keys(Keys.PAGE_UP)
        except Exception as error:
            if 'NoSuchElementException' in str(type(error)):
                pass
            

Dados de jogos coletados: 260754
saved file: data\22_02_17_with_147_events.csv
Dados de jogos coletados: 260901
saved file: data\21_02_17_with_28_events.csv
Dados de jogos coletados: 260929
problema em abrir os eventos
Dados de jogos coletados: 260929
Dados de jogos coletados: 260929
problema em abrir os eventos
Dados de jogos coletados: 260929
saved file: data\20_02_17_with_69_events.csv
Dados de jogos coletados: 260998
saved file: data\19_02_17_with_29_events.csv
Dados de jogos coletados: 261027
problema em abrir os eventos
Dados de jogos coletados: 261027
saved file: data\18_02_17_with_706_events.csv
Dados de jogos coletados: 261733
saved file: data\17_02_17_with_25_events.csv
Dados de jogos coletados: 261758
problema em abrir os eventos
Dados de jogos coletados: 261758
problema em abrir os eventos
Dados de jogos coletados: 261758
Dados de jogos coletados: 261758
problema em abrir os eventos
Dados de jogos coletados: 261758
problema em abrir os eventos
Dados de jogos coletados: 2617

33

In [ ]:
#total de jogos coletados
qntd = []
for game in os.listdir(r'data'):
    qntd.append(int(game.split('_')[-2]))
    
print(sum(qntd))